# Testing the TaylorAnalysis Framework

In this notebook we will demonstrate the correctness of the taylorcoefficients computed by the TaylorAnalysis based on a simple example.

In [1]:
import tayloranalysis as ta
import torch

First setup a simple polynomial function 

$$f(x,y) = xy+xy^2+y^3$$ 

Since the TaylorAnalysis uses `torch`, we'll define the polynomial within a `torch.nn.Module`.

In [2]:
class Polynom(torch.nn.Module):
    def __init__(self):
        torch.nn.Module.__init__(self)

    def forward(self, point):
        y = point[:,1]
        x = point[:,0]
        return x*y + x * y**2 + y**3

Initialize model and extend it with the TaylorAnalysis module.

In [3]:
model = Polynom()
model = ta.extend_model(model)

Remember the taylor series for a function $f(x, y)$ at point $a$:
$$f(x, y) = \sum_{m=0}^{\infty} \sum_{n=0}^{\infty} \frac{1}{m!n!} \left[\frac{\partial^{m+n}f}{\partial x^m \partial y^n}(x, y)(x-a)^m(y-b)^n\right]$$

with the derivative orders $m$, $n$ w.r.t. the input variables $x$ and $y$.


The corresponding taylor coefficients are 
$$\mathrm{TC}_\mathrm{m,n}(x,y) = \frac{1}{m!n!} \frac{\partial^{m+n}f}{\partial x^m \partial y^n}(x, y)$$



Now choose some taylorcoefficients we want to look at and define a point to expand around:

In [4]:
combinations = [(0,), (0,0), (0,1,1), (1,), (0,1), (0,0,1), (1,1,1)]
point = torch.tensor([[3,2]]).float()

Loop over combinations and determine the corresponding taylorcoefficients.
As reduction method we will use the default identity function.

In [5]:
for combi in combinations:
    
    # compute taylorcoefficient based on point
    tc = model.get_tc(point, [combi])
    tc = list(tc.values())[0]
    
    # determine derivation orders
    xorder = combi.count(0)
    yorder = combi.count(1)

    # print result
    print(f"TC{chr(8320 + xorder)}{chr(8320 + yorder)}({int(point[0][0])},{int(point[0][1])}) = {float(tc)}")

TC₁₀(3,2) = 6.0
TC₂₀(3,2) = 0.0
TC₁₂(3,2) = 1.0
TC₀₁(3,2) = 27.0
TC₁₁(3,2) = 5.0
TC₂₁(3,2) = 0.0
TC₀₃(3,2) = 1.0


This can be manually compared with the corresponding derivations of

$$f(x,y) = xy+xy^2+y^3$$ 

Therefore holds for TC₁₀(3,2):

$$
\begin{align}
\mathrm{TC}_\mathrm{1,0}(3,2) &= \frac{1}{1!0!} \frac{\partial^{1+0}f}{\partial x^1 \partial y^0}(x, y)|_{x=3, y=2} \\
&= \frac{\partial}{\partial x}f(x, y) |_{x=3, y=2} \\
&= \frac{\partial}{\partial x} \left(xy+xy^2+y^3\right)|_{x=3, y=2} \\
&= y+y^2|_{x=3, y=2} \\
&= 6
\end{align}
$$

... and same for remaining.
